In [4]:
library(sits)
set.seed(123)

ERROR: Error in library(sits): there is no package called ‘sits’


### Create cube

In [ ]:
#
# General definitions
#

# Dates
start_date <- "2022"

# Cube directory
cube_dir <- "data/output/cube/"

# Bands
cube_bands <- c(
  "BLUE", "EVI", "GREEN", "NDVI", "NIR08", "RED", "SWIR16", "SWIR22", "CLOUD"
)

# Hardware - multicores
multicores <- 24

#
# 1. Create cube directory
#
dir.create(cube_dir, recursive = TRUE)

#
# 2. Read ROI
#
tile <- "006008"

#
# 3. Generate cubes
#
# define dates
start_date <- paste0(y, "-01-01")
end_date <- paste0(y, "-12-31")

# load cube
cube <- sits_cube(
source     = "BDC",
collection = "LANDSAT-OLI-16D",
tiles      = tile,
start_date = start_date,
end_date   = end_date,
bands      = cube_bands
)

# define cube directory
cube_dir_year <- paste0(cube_dir, y)

# create cube directory
dir.create(cube_dir_year, recursive = TRUE)

# regularize cube
reg_cube <- sits_regularize(
cube       = cube,
period     = "P1M",
res        = 30,
tiles      = cube$tile,
multicores = multicores,
output_dir = cube_dir_year
)

### Reading Samples

In [ ]:
set.seed(123)
library(sits)

#
# General definition
#

# Samples
samples_file <- "data/raw/samples/samples.rds"

# Output directory
base_output_dir <- "data/output/"

#
# 1. Load samples
#
samples <- readRDS(samples_file)

In [ ]:
plot(sits_patterns(samples))

### Tunning CNN

In [ ]:
#
# 2. Train model
#
tuned_tempcnn <- sits_tuning(
  samples   = samples,
  ml_method = sits_tempcnn(),
  params        = sits_tuning_hparams(
    optimizer   = torch::optim_adamw,
    cnn_kernels = choice(c(3, 3, 3), c(5, 5, 5), c(7, 7, 7)),
    cnn_layers  = choice(c(2^5, 2^5, 2^5), c(2^6, 2^6, 2^6), c(2^7, 2^7, 2^7)),
    opt_hparams = list(
      lr           = loguniform(10^-2, 10^-4),
      weight_decay = loguniform(10^-2, 10^-8)
    )
  ),
  trials     = 20,
  multicores = 6,
  progress   = TRUE
)

In [ ]:
tuned_tempcnn

### Training CNN

In [1]:
tcnn_model <- sits_train(
    samples, sits_tempcnn()
)

#
# 3. Save model
#

# directory
model_dir <- paste0(base_output_dir, "/model")  

# create directory
dir.create(model_dir, recursive = TRUE)

# save
saveRDS(tcnn_model, paste0(model_dir,"/model_tcnn_tuned.rds"))

ERROR: Error in sits_train(samples, sits_tempcnn()): could not find function "sits_train"


### Tuning LightTAE

In [ ]:
tuned_lighttae <- sits_tuning(
  samples   = samples,
  ml_method = sits_lighttae(),
  params        = sits_tuning_hparams(
    optimizer   = torch::optim_adamw,
    opt_hparams = list(
      lr           = loguniform(10^-2, 10^-4),
      weight_decay = loguniform(10^-2, 10^-8)
    )
  ),
  trials     = 20,
  multicores = 6,
  progress   = TRUE
)

### Training LightTAE

In [ ]:
lighttae_model <- sits_train(
    samples, sits_lighttae()
)

#
# 3. Save model
#

# directory
model_dir <- paste0(base_output_dir, "/model")  

# create directory
dir.create(model_dir, recursive = TRUE)

# save
saveRDS(lighttae_model, paste0(model_dir,"/model_lighttae_tuned.rds"))

### Pixel-based classification - TCNN

In [ ]:
# Output directory
output_dir <- "data/output/classifications/tccn"

# Version
results_version <- "tcnn-8cls-v17-yr-6bands-w7-nf05-s10-clean"

#
# 1. Create directory
#
dir.create(output_dir, recursive = TRUE)


#
# 4. Classify
#
cube_dir_year <- paste0(cube_dir, y)
start_date <- paste0(y, "-01-01")
end_date <- paste0(y, "-12-31")

# classify
out_dir <- fs::path(output_dir) / y
fs::dir_create(out_dir, recurse = TRUE)

probs_cube <- sits_classify(
    data       = cube,
    ml_model   = tcnn_model,
    memsize    = 54,
    gpu_memory = 10,
    multicores = 24,
    output_dir = out_dir,
    version    = results_version,
    progress   = TRUE,
    verbose    = TRUE
)

# smooth
probs_bayes <- sits_smooth(
    cube           = probs_cube,
    window_size    = 7,
    neigh_fraction = 0.5,
    smoothness     = c(10, 10, 10, 10, 10, 10, 10, 10, 10),
    memsize        = 60,
    multicores     = 24,
    output_dir     = out_dir,
    version        = results_version
)

# generate map
class_cube <- sits_label_classification(
    cube       = probs_bayes,
    memsize    = 60,
    multicores = 24,
    output_dir = out_dir,
    version    = results_version
)

# clean map
out_clean <- fs::path(out_dir) / "clean"
fs::dir_create(out_clean, recurse = TRUE)

clean_tile <- sits_clean(
    cube         = class_cube,
    window_size  = 7,
    memsize      = 60,
    multicores   = 24,
    output_dir   = out_clean,
    version      = results_version,
    progress     = TRUE
)

### Pixel-based classification - LightTAE

In [ ]:
# Output directory
output_dir <- "data/output/classifications/lighttae"

# Version
results_version <- "lighttaeb-8cls-v17-yr-6bands-w7-nf05-s10-clean"

#
# 1. Create directory
#
dir.create(output_dir, recursive = TRUE)


#
# 4. Classify
#
cube_dir_year <- paste0(cube_dir, y)
start_date <- paste0(y, "-01-01")
end_date <- paste0(y, "-12-31")

# classify
out_dir <- fs::path(output_dir) / y
fs::dir_create(out_dir, recurse = TRUE)

probs_cube <- sits_classify(
    data       = cube,
    ml_model   = lighttae_model,
    memsize    = 54,
    gpu_memory = 10,
    multicores = 24,
    output_dir = out_dir,
    version    = results_version,
    progress   = TRUE,
    verbose    = TRUE
)

# smooth
probs_bayes <- sits_smooth(
    cube           = probs_cube,
    window_size    = 7,
    neigh_fraction = 0.5,
    smoothness     = c(10, 10, 10, 10, 10, 10, 10, 10, 10),
    memsize        = 60,
    multicores     = 24,
    output_dir     = out_dir,
    version        = results_version
)

# generate map
class_cube <- sits_label_classification(
    cube       = probs_bayes,
    memsize    = 60,
    multicores = 24,
    output_dir = out_dir,
    version    = results_version
)

# clean map
out_clean <- fs::path(out_dir) / "clean"
fs::dir_create(out_clean, recurse = TRUE)

clean_tile <- sits_clean(
    cube         = class_cube,
    window_size  = 7,
    memsize      = 60,
    multicores   = 24,
    output_dir   = out_clean,
    version      = results_version,
    progress     = TRUE
)